Knowledge distillation is a technique used to transfer knowledge from a larger, more complex model (teacher) to a smaller, simpler model (student). To distill knowledge from one model to another, we take a pre-trained teacher model trained on a certain task (image classification for this case) and randomly initialize a student model to be trained on image classification. Next, we train the student model to minimize the difference between it’s outputs and the teacher’s outputs, thus making it mimic the behavior. It was first introduced in a paper by Hinton, Vinyals and Dean: https://arxiv.org/abs/1503.02531. 

In this guide, we will do task-specific knowledge distillation. Specifically, this guide demonstrates how you can distill a fine-tuned ViT model (teacher model) to a MobileNet (student model) using the Trainer API of HF Transformers.

# Libraries

In [ ]:
pip install transformers datasets accelerate tensorboard evaluate --upgrade

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from datasets import load_dataset
from transformers import AutoImageProcessor, TrainingArguments, Trainer

# Data Load

In [ ]:
# Load beans dataset. We'll use merve/beans-vit-224 model as teacher model. 
# It’s an image classification model, based on google/vit-base-patch16-224-in21k fine-tuned on beans dataset
dataset = load_dataset("beans")

# Preprocessing

In [ ]:
# We can use a preprocessor from either model

teacher_processor = AutoImageProcessor.from_pretrained("merve/beans-vit-224")

def process(examples):
    processed_inputs = teacher_processor(examples["image"])
    return processed_inputs

# use the map() method of dataset to apply the preprocessing to every split of the dataset
processed_datasets = dataset.map(process, batched=True)

In [ ]:
# We will distill merve/beans-vit-224 model to a randomly initialized MobileNetV2
# We want the randomly initialized MobileNet to mimic the teacher model
# To achieve this, (1) we first get the logits output from the teacher and the student, 
# (2) we divide each of them by the parameter temperature which controls the importance of each soft target,
# (3) use the Kullback-Leibler Divergence loss to compute the divergence between the student and teacher.
# If the logits from the two models are identical, their KL divergence will equal zero.

class ImageDistilTrainer(Trainer):
    def __init__(self, teacher_model=None, student_model=None, temperature=None, lambda_param=None,  *args, **kwargs):
        super().__init__(model=student_model, *args, **kwargs)
        self.teacher = teacher_model
        self.student = student_model
        self.loss_function = nn.KLDivLoss(reduction="batchmean")
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.teacher.to(device)
        self.teacher.eval()
        self.temperature = temperature
        self.lambda_param = lambda_param

    def compute_loss(self, student, inputs, return_outputs=False):
        student_output = self.student(**inputs)

        with torch.no_grad():
            teacher_output = self.teacher(**inputs)

        # Compute soft targets for teacher and student
        soft_teacher = F.softmax(teacher_output.logits / self.temperature, dim=-1)
        soft_student = F.log_softmax(student_output.logits / self.temperature, dim=-1)

        # Compute the loss
        distillation_loss = self.loss_function(soft_student, soft_teacher) * (self.temperature ** 2)

        # Compute the true label loss
        student_target_loss = student_output.loss

        # Calculate final loss
        loss = (1. - self.lambda_param) * student_target_loss + self.lambda_param * distillation_loss
        return (loss, student_output) if return_outputs else loss